In [1]:
from asset_vulnerability_report.settings import DATASET_FOLDER
from glob import glob
from os.path import join

source_paths = sorted(glob(join(DATASET_FOLDER, '*.json')))
source_path = source_paths[-3]
source_path

'/home/rhh/Experiments/asset-vulnerability-report/datasets/nvdcve-1.0-2017.json'

In [4]:
import json
d = json.load(open(source_path))

In [5]:
d.keys()

dict_keys(['CVE_data_type', 'CVE_data_format', 'CVE_data_version', 'CVE_data_numberOfCVEs', 'CVE_data_timestamp', 'CVE_Items'])

In [7]:
i = d['CVE_Items'][0]

# ID

In [23]:
i['cve']['CVE_data_meta']['ID']

'CVE-2017-0001'

# URIs

In [50]:
from itertools import chain

def extract_cpe_matches(node):
    if 'cpe_match' in node:
        return node['cpe_match']
    if 'children' in node:
        return chain(*[extract_cpe_matches(_) for _ in node['children']])
    return []

In [55]:
uris = []
for node in i['configurations']['nodes']:
    for cpe_match in extract_cpe_matches(node):
        uris.append(cpe_match['cpe23Uri'][8:])
uris

['o:microsoft:windows_10:-:*:*:*:*:*:*:*',
 'o:microsoft:windows_10:1511:*:*:*:*:*:*:*',
 'o:microsoft:windows_10:1607:*:*:*:*:*:*:*',
 'o:microsoft:windows_7:*:sp1:*:*:*:*:*:*',
 'o:microsoft:windows_8.1:*:*:*:*:*:*:*:*',
 'o:microsoft:windows_rt_8.1:*:*:*:*:*:*:*:*',
 'o:microsoft:windows_server_2008:*:sp2:*:*:*:*:*:*',
 'o:microsoft:windows_server_2008:r2:sp1:*:*:*:*:*:*',
 'o:microsoft:windows_server_2012:-:*:*:*:*:*:*:*',
 'o:microsoft:windows_server_2012:r2:*:*:*:*:*:*:*',
 'o:microsoft:windows_vista:*:sp2:*:*:*:*:*:*']

# Text

In [45]:
texts = []
for description in i['cve']['description']['description_data']:
    if description['lang'] != 'en':
        continue
    texts.append(description['value'])
texts

['The Graphics Device Interface (GDI) in Microsoft Windows Vista SP2; Windows Server 2008 SP2 and R2 SP1; Windows 7 SP1; Windows 8.1; Windows Server 2012 Gold and R2; Windows RT 8.1; and Windows 10 Gold, 1511, and 1607 allows local users to gain privileges via a crafted application, aka "Windows GDI Elevation of Privilege Vulnerability." This vulnerability is different from those described in CVE-2017-0005, CVE-2017-0025, and CVE-2017-0047.']

In [47]:
assert len(texts) == 1

# Date

In [40]:
from datetime import datetime
datetime.strptime(i['publishedDate'], '%Y-%m-%dT%H:%MZ')

datetime.datetime(2017, 3, 17, 0, 59)

# Score

In [21]:
i['impact']['baseMetricV3']['cvssV3']['baseScore']

In [20]:
i['impact']['baseMetricV2']['cvssV2']['baseScore']

7.2